# Converting IOData Information into Chemical JSON Forma

## Introduction
IOData is a python utility for parsing and converting common chemical 

The goal of this tool is to convert IOData into Chemical JSON, a file format increasingly used by the OpenChem Group

## Dependencies
1. [IOData](https://iodata.readthedocs.io/en/latest/index.html#) 
2. [Numpy](https://numpy.org/)
3. [Python-JSON](https://docs.python.org/3/library/json.html#module-json)
4. [ChemicalJSON](https://wiki.openchemistry.org/Chemical_JSON)
5. [JSON Encoder](https://stackoverflow.com/questions/57269741/typeerror-object-of-type-ndarray-is-not-json-serializable)

## Limitations of File Formats
Per IOData documentation, certain file formats cannot read certain features via IOData, an example being the inability to use `bonds` to obtain bond data from `fchk` files 

In [20]:
import iodata
from iodata import load_one, dump_one
import numpy as np

import json

## IOData Data Parsing 

1. We use `load_one` to load our input file
2. We also want to initialize our chemical JSON by creating a dictionary starting with element `"chemicaljson":0` 
3. We now want to populate our dictionary, `molDict`, by using IOData API calls, such as 
    - `title` - obtain the name of the molecule, including information such as the computational basis set used, and the hybridization state
    - `atcoords` - obtain 3D XYZ Cartesian Coordinates
    - `atnums` - obtain the atomic number of each atom in the system
    - `atcharges` - obtain a dictionary detailing the charges of each atom in the system 


In [30]:
mol = load_one("dichloropyridine26_q+0.fchk")

molDict = {"chemicaljson":0} 

name = mol.title
molDict.update({'title':name})

print(molDict["title"])

print(" ")

molCoord = np.asarray(mol.atcoords)

molDict.update({'coords':
                {'3d':molCoord}
               })
        

molAtomicNums = mol.atnums
molDict.update({'atoms':
                {'elements':
                {'numbers':molAtomicNums}
               }}) 


print(" ")

# Sanity Check
print(molDict)

dichloropyridine26_q+0 ub3lyp/cc-pvtz sp-stable
 
 
{'chemicaljson': 0, 'title': 'dichloropyridine26_q+0 ub3lyp/cc-pvtz sp-stable', 'coords': {'3d': array([[-4.95130924e+00,  2.31173977e+00, -2.64561659e-04],
       [ 4.95149821e+00,  2.31139962e+00,  2.07869875e-04],
       [ 7.55890453e-05,  1.84250188e+00, -1.88972613e-05],
       [-9.44863066e-05, -3.41775868e+00,  1.13383568e-04],
       [-2.28277027e+00, -2.09580077e+00, -1.88972613e-05],
       [ 2.28267578e+00, -2.09595195e+00,  1.70075352e-04],
       [-2.13763930e+00,  5.39856962e-01, -3.77945227e-05],
       [ 2.13769600e+00,  5.39705784e-01,  7.55890453e-05],
       [-1.70075352e-04, -5.47831606e+00,  1.51178091e-04],
       [-4.10063012e+00, -3.05289036e+00, -7.55890453e-05],
       [ 4.10047894e+00, -3.05315492e+00,  2.83458920e-04]])}, 'atoms': {'elements': {'numbers': array([17, 17,  7,  6,  6,  6,  6,  6,  1,  1,  1])}}}


## Encoding 



In [28]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32,
                              np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

## Output to JSON 

We now want to output our dictionary that we populated with IOData, and encoded using the afforementioned JSONEncoder. We accomplish this using `json.dumps`, then creating a new writeable file, in which the JSON jump is written into. 

As an intermediary sanity check, we can examine our JSON using `print(molJson)` 

In [29]:
molJson = json.dumps(molDict, indent=3, cls=NumpyEncoder)
print(molJson)

f = open("dichloropyridine26_q+0.json", "w")
f.write(molJson)
f.close()



{
   "chemicaljson": 0,
   "title": "dichloropyridine26_q+0 ub3lyp/cc-pvtz sp-stable",
   "coords": {
      "3d": [
         [
            -4.95130924,
            2.31173977,
            -0.000264561659
         ],
         [
            4.95149821,
            2.31139962,
            0.000207869875
         ],
         [
            7.55890453e-05,
            1.84250188,
            -1.88972613e-05
         ],
         [
            -9.44863066e-05,
            -3.41775868,
            0.000113383568
         ],
         [
            -2.28277027,
            -2.09580077,
            -1.88972613e-05
         ],
         [
            2.28267578,
            -2.09595195,
            0.000170075352
         ],
         [
            -2.1376393,
            0.539856962,
            -3.77945227e-05
         ],
         [
            2.137696,
            0.539705784,
            7.55890453e-05
         ],
         [
            -0.000170075352,
            -5.47831606,
            0.000